<a href="https://colab.research.google.com/github/dhruvthanki/CISC642/blob/main/PR3_1a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy

In [ ]:
data_transforms = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
trainset = torchvision.datasets.CIFAR100(root='./data', train = True, transform = data_transforms, download=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16,shuffle=True, num_workers=2)

Files already downloaded and verified


In [ ]:
model = models.vgg16(pretrained=True)

num_ftrs=model.classifier[6].in_features
num_cls = len(trainset.classes)
model.classifier[6] = nn.Linear(num_ftrs, num_cls)

for param in model.parameters(): # freeze
  param.requires_grad = False

for param in model.classifier[6].parameters(): # train the last linear layer.
  param.requires_grad = True

In [ ]:
num_epochs = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
train = int(input("Enter 1 to train otherwise 0:"))
if train == 1:
  model.train()
  for epoch in range(num_epochs):  # loop over the dataset multiple times

      running_loss = 0.0
      for i, data in enumerate(trainloader, 0):
          # get the inputs
          inputs, labels = data
          inputs, labels = inputs.cuda(), labels.cuda()

          # zero the parameter gradients
          optimizer.zero_grad()

         # forward + backward + optimize
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # print statistics
          running_loss += loss.item()
          if i % 2000 == 1999:    # print every 2000 mini-batches
              print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
              running_loss = 0.0
              
  print('Finished Training')
  best_model_wts = copy.deepcopy(model.state_dict())
  torch.save(best_model_wts , '/content/drive/MyDrive/UD/Courses/CV/PR3/best_model_weight_PR3_1a.pth')
else:
  model.load_state_dict(torch.load('/content/drive/MyDrive/UD/Courses/CV/PR3/best_model_weight_PR3_1a.pth'))
  model.eval()

Enter 1 to train otherwise 0:0


In [ ]:
testset = torchvision.datasets.CIFAR100(root='./data', train=False, transform = data_transforms, download=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,shuffle=True, num_workers=2)

Files already downloaded and verified


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 57 %
